In [1]:

# Ragas evaluation on pre-defined examples

%pip install ragas datasets python-dotenv

from dotenv import load_dotenv
import os
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, context_precision, context_recall, answer_relevancy

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in .env file.")


qa_pairs = [
    (
        "What is the season for leaves to turn color?",
        "The season when leaves turn color is autumn (fall). In most parts of the Northern Hemisphere, "
        "this typically happens from late September through October, sometimes into November. "
        "It's caused by shorter days and cooler temperatures breaking down chlorophyll, revealing reds, oranges, and yellows."
    ),
    (
        "Why do we have seasons?",
        "We have seasons because the Earth’s axis is tilted relative to its orbit around the Sun. "
        "This tilt causes different parts of Earth to receive varying amounts of sunlight during the year."
    ),
    (
        "Why do we have seasons?",
        "Because of how Earth moves around the Sun."
    ),
    (
        "What is the season for leaves to turn color?",
        "Leaves turn color when it gets colder."
    ),
    (
        "Why do we have seasons?",
        "Because the Moon’s gravity changes how the Earth spins throughout the year."
    ),
    (
        "What is the season for leaves to turn color?",
        "Fish migrate upstream to lay eggs."
    )
]

# run eval

for idx, (question, response) in enumerate(qa_pairs, start=1):
    context = response
    ground_truth = response
    
    print(f"\n=== Example {idx} ===")
    print(f"Question: {question}")
    print(f"Response: {response}")
    
    ragas_data = Dataset.from_dict({
        "question": [question],
        "contexts": [[context]],
        "answer": [response],
        "ground_truths": [[ground_truth]],
        "reference": [ground_truth]
    })
    
    results = evaluate(
        ragas_data,
        metrics=[faithfulness, context_precision, context_recall, answer_relevancy],
        column_map={
            "question": "question",
            "contexts": "contexts",
            "answer": "answer",
            "ground_truths": "ground_truths",
            "reference": "reference"
        }
    )
    
    # Print metrics 
    for metric, value in results.scores[0].items():
        print(f"{metric:<18}: {float(value):.3f}")


Note: you may need to restart the kernel to use updated packages.

=== Example 1 ===
Question: What is the season for leaves to turn color?
Response: The season when leaves turn color is autumn (fall). In most parts of the Northern Hemisphere, this typically happens from late September through October, sometimes into November. It's caused by shorter days and cooler temperatures breaking down chlorophyll, revealing reds, oranges, and yellows.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.938

=== Example 2 ===
Question: Why do we have seasons?
Response: We have seasons because the Earth’s axis is tilted relative to its orbit around the Sun. This tilt causes different parts of Earth to receive varying amounts of sunlight during the year.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.974

=== Example 3 ===
Question: Why do we have seasons?
Response: Because of how Earth moves around the Sun.


...

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.912

=== Example 5 ===
Question: Why do we have seasons?
Response: Because the Moon’s gravity changes how the Earth spins throughout the year.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.822

=== Example 6 ===
Question: What is the season for leaves to turn color?
Response: Fish migrate upstream to lay eggs.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 0.000
context_recall    : 1.000
answer_relevancy  : 0.748


In [2]:


from dotenv import load_dotenv
import os
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, context_precision, context_recall, answer_relevancy

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in .env file.")

examples = [
    {
        "question": "Why do leaves change color in the fall?",
        "contexts": [
            "Leaves turn color in autumn due to changes in daylight and temperature.",
            "Photosynthesis slows and chlorophyll breaks down, revealing red and yellow pigments.",
            "Some tree species have different pigment compounds that show up prominently in fall."
        ],
        "response": "Leaves change color in autumn because shorter days and cooler temperatures cause chlorophyll to break down, revealing other pigments.",
        "reference": "Leaves change color in autumn because chlorophyll breaks down due to shorter days and cooler weather."
    },
    {
        "question": "Why do we have seasons on Earth?",
        "contexts": [
            "The tilt of Earth's axis relative to its orbit causes different parts of Earth to receive varying amounts of sunlight throughout the year.",
            "This axial tilt is approximately 23.5 degrees.",
            "Changes in distance from the sun during the orbit do not cause seasons."
        ],
        "response": "Seasons occur because the Earth's axis is tilted, which changes how sunlight hits different parts of the planet throughout the year.",
        "reference": "Earth has seasons because its axis is tilted relative to its orbital plane, causing varying sunlight."
    },
    {
        #  incomplete
        "question": "Why do we have seasons on Earth?",
        "contexts": [
            "Earth revolves around the Sun in an elliptical orbit.",
            "There are four main seasons: spring, summer, autumn, and winter."
        ],
        "response": "We have seasons because Earth moves around the Sun.",
        "reference": "Earth has seasons because its axis is tilted relative to its orbit, changing sunlight."
    },
    {
        # unrelated / wrong
        "question": "Why do leaves change color in the fall?",
        "contexts": [
            "Fish migrate upstream to spawn in freshwater rivers.",
            "This migration ensures the survival of the next generation."
        ],
        "response": "Leaves change color because fish swim upstream in autumn.",
        "reference": "Leaves change color due to breakdown of chlorophyll in cooler temperatures."
    }
]


for idx, ex in enumerate(examples, start=1):
    print(f"\n=== Example {idx} ===")
    print(f"Question: {ex['question']}")
    print(f"Response: {ex['response']}")
    
    ragas_data = Dataset.from_dict({
        "question": [ex['question']],
        "contexts": [ex['contexts']],
        "answer": [ex['response']],
        "ground_truths": [[ex['reference']]],
        "reference": [ex['reference']]
    })
    
    results = evaluate(
        ragas_data,
        metrics=[faithfulness, context_precision, context_recall, answer_relevancy],
        column_map={
            "question": "question",
            "contexts": "contexts",
            "answer": "answer",
            "ground_truths": "ground_truths",
            "reference": "reference"
        }
    )
    
    for metric, value in results.scores[0].items():
        print(f"{metric:<18}: {float(value):.3f}")



=== Example 1 ===
Question: Why do leaves change color in the fall?
Response: Leaves change color in autumn because shorter days and cooler temperatures cause chlorophyll to break down, revealing other pigments.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.987

=== Example 2 ===
Question: Why do we have seasons on Earth?
Response: Seasons occur because the Earth's axis is tilted, which changes how sunlight hits different parts of the planet throughout the year.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 1.000
context_recall    : 1.000
answer_relevancy  : 0.970

=== Example 3 ===
Question: Why do we have seasons on Earth?
Response: We have seasons because Earth moves around the Sun.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 1.000
context_precision : 0.000
context_recall    : 0.000
answer_relevancy  : 0.974

=== Example 4 ===
Question: Why do leaves change color in the fall?
Response: Leaves change color because fish swim upstream in autumn.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

faithfulness      : 0.000
context_precision : 0.000
context_recall    : 0.000
answer_relevancy  : 0.000
